In [ ]:
# simple p
%%sql
with seg_all_cnt as (
  select 
    seg_id,
    count(*) as all_cnt
  from seg_users where seg_id in () group by seg_id
), seg_p as (
  select 
    l.seg_id,
    r2.id,
    r2.name
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c 
        where created_at between '' and '') as l
    inner join seg_users on l.device_id = seg_users.device_id and seg_users.seg_id in ()
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    inner join seg_all_cnt as r3 on l.seg_id = r3.seg_id
  group by l.seg_id, r2.id, r2.name
), res_id as (

), res_all_cnt as (
    select
      count(*) as all_cnt
    from res_id group by seg_id
), res_p as (
  select
    l.seg_id,
    r2.id,
    r2.name,
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c where created_at between '' and '') as l
    inner join res_id as r on l.device_id = r.device_id
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    , res_all_cnt as r3
)
select
  l.seg_id,
  l.id,
  l.name,
  l.p_cnt / r.p_cnt as p
from seg_p as l
inner join res_p as r l.seg_id = r.seg_id
group by l.seg_id, l.id, l.name

In [ ]:
# normarized p
%%sql
with seg_all_cnt as (
  select 
    seg_id,
    count(*) as all_cnt
  from seg_users where seg_id in () group by seg_id
), seg_p as (
  select 
    r3.seg_id,
    r2.id,
    r2.name
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c 
        where created_at between '' and '') as l
    inner join seg_users on l.device_id = seg_users.device_id and seg_users.seg_id in ()
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    inner join seg_all_cnt as r3 on l.seg_id = r3.seg_id
  group by l.seg_id, r2.id, r2.name
), seg_p_norm as (
  select
    seg_id,
    id,
    name,
    p_cnt,
    (p_cnt - avg(p_cnt) over (partition by seg_id)) / stddev() over (partition by seg_id) as p_norm
  from seg_p
), res_id as (

), res_all_cnt as (
    select
      count(*) as all_cnt
    from res_id group by seg_id
), res_p as (
  select
    r2.id,
    r2.name,
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c where created_at between '' and '') as l
    inner join res_id as r on l.device_id = r.device_id
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    , res_all_cnt as r3
), res_p_norm (
  select
    id,
    name,
    p_cnt
    (p_cnt - avg(p_cnt) over (partition by seg_id)) / stddev() over (partition by seg_id) as p_norm
)
select
  l.seg_id,
  l.id,
  l.name,
  l.p_cnt / r.p_cnt as p
  l.p_cnt_norm / r.p_cnt_norm as p_norm
from seg_p_norm as l
inner join res_p_norm as r l.seg_id = r.seg_id
group by l.seg_id, l.id, l.name

In [ ]:
# KLD
%%sql
with seg_all_cnt as (
  select 
    seg_id,
    count(*) as all_cnt
  from seg_users where seg_id in () group by seg_id
), seg_p as (
  select 
    r3.seg_id,
    r2.id,
    r2.name
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c 
        where created_at between '' and '') as l
    inner join seg_users on l.device_id = seg_users.device_id and seg_users.seg_id in ()
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    inner join seg_all_cnt as r3 on l.seg_id = r3.seg_id
  group by l.seg_id, r2.id, r2.name
), seg_p_norm as (
  select
    seg_id,
    id,
    name,
    p_cnt,
    (p_cnt - avg(p_cnt) over (partition by seg_id)) / stddev() over (partition by seg_id) as p_norm
  from seg_p
), res_id as (

), res_all_cnt as (
    select
      count(*) as all_cnt
    from res_id group by seg_id
), res_p as (
  select
    r2.id,
    r2.name,
    count(*) / r3.all_cnt * 100.0 as p_cnt
  from (select * from h_u_c where created_at between '' and '') as l
    inner join res_id as r on l.device_id = r.device_id
    inner join region_p as r1 on l.key = r1.p_id
    inner join r as r2 on r1.r_id = r2.id and r2.off = 1
    , res_all_cnt as r3
), res_p_norm (
  select
    id,
    name,
    p_cnt
    (p_cnt - avg(p_cnt) over (partition by seg_id)) / stddev() over (partition by seg_id) as p_norm
)
select
  l.seg_id,
  l.id,
  l.name,
  l.p_cnt / r.p_cnt as p
  l.p_cnt_norm / r.p_cnt_norm as p_norm
  l.p_cnt * log(l.p_cnt / r.p_cnt) as kl
  l.p_cnt_norm * log(l.p_cnt_norm / r.p_cnt_norm) as kl_norm
from seg_p_norm as l
inner join res_p_norm as r l.seg_id = r.seg_id
group by l.seg_id, l.id, l.name